In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

## Model

In [4]:
model = BaseModel(opt, build_model(opt)).train().cuda()

In [5]:
model

BaseModel(
  (model): CDNHOI(
    (backbone): D2FocalNet(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): FocalModulationBlock(
              (dw1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (modulation): FocalModulation(
                (f): Linear(in_features=96, out_features=196, bias=True)
                (h): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
                (act): GELU(approximate='none')
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (focal_layers): Mo

In [6]:
t = []
t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
transform = transforms.Compose(t)
pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()
image_pth = '../images/animals.png'

with torch.no_grad():
    image_ori = Image.open(image_pth).convert('RGB')
    width = image_ori.size[0]
    height = image_ori.size[1]
    image = transform(image_ori)
    image = np.asarray(image)
    image_ori = np.asarray(image_ori)
    images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
    batch_inputs = [{'image': images, 'height': height, 'width': width}]
    out = model.forward(batch_inputs)
    print(out)

Calling forward() may cause unpredicted behavior of PixelDecoder module.


{'pred_obj_logits': tensor([[[ 0.2708,  0.8102,  0.0560,  ...,  0.1979, -0.2894,  0.5397],
         [ 0.4842,  0.9740,  0.0050,  ..., -0.0348, -0.5324,  0.5344],
         [ 0.4247,  1.3230,  0.0921,  ...,  0.0657, -0.7571,  0.3034],
         ...,
         [ 0.3730,  0.7890, -0.0562,  ..., -0.0504, -0.4359,  0.2787],
         [ 0.7168,  0.9968, -0.0635,  ...,  0.3093, -0.3467,  0.3416],
         [ 0.4348,  0.7422, -0.1290,  ...,  0.4767, -0.3645,  0.3547]]],
       device='cuda:0'), 'pred_verb_logits': tensor([[[-0.8402, -0.0145, -1.1549,  ..., -0.7912, -0.7599,  0.5445],
         [-0.9658, -0.2393, -1.2318,  ..., -0.8057, -0.4699,  0.5005],
         [-0.5325, -0.2734, -1.1044,  ..., -0.7634, -0.5131,  0.4343],
         ...,
         [-0.5592, -0.2885, -1.0356,  ..., -0.8988, -0.3973,  0.7178],
         [-0.6580,  0.0652, -1.0405,  ..., -1.0146, -0.5080,  0.7786],
         [-0.7376, -0.1507, -0.9025,  ..., -0.8933, -0.3404,  0.4627]]],
       device='cuda:0'), 'pred_sub_boxes': tensor([